In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#Had to manually run these from within the Sagemaker notebook venv
#!apt-get update
#!python3 -m pip install --upgrade pip
#!pip install opencv-python
#!apt-get install -y libglib2.0-0
#!apt-get install -y libsm6 libxext6 libxrender-dev
#!pip install matplotlib --upgrade
#!apt-get install -y python3-skimage
#!pip install scikit-image==0.16.2

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report


# Root directory of the project
ROOT_DIR = os.path.abspath(".")
print(ROOT_DIR)
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from src.VehiclesDataset import VehiclesDataset

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'models')

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, 'models/mask_rcnn_coco.h5')
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

## Configuration

In [6]:
class VehiclesConfig(Config):
    NAME = "vehicles"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    NUM_CLASSES = 1 + 2  # background + 2 shapes
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256
    #BACKBONE = "resnet50" -- default is resnet101

    #These are for BW
    #IMAGE_CHANNEL_COUNT = 1
    #MEAN_PIXEL = 1
    
    #Faster epochs?
    #STEPS_PER_EPOCH = 100
config = VehiclesConfig()
#config.display()

## Create training and validation datasets

In [7]:
train_dataset_dir = '../../data/color/train'
val_dataset_dir = '../../data/color/val'
mask_dir = '../../data/masks/'


# Training dataset
dataset_train = VehiclesDataset()
dataset_train.load_images(train_dataset_dir, mask_dir)
dataset_train.prepare()

# Validation dataset
dataset_val = VehiclesDataset()
dataset_val.load_images(val_dataset_dir, mask_dir)
dataset_val.prepare()

In [13]:
class InferenceConfig(VehiclesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = os.path.join(MODEL_DIR, "mask_rcnn_vehicles-clean-start-5.h5")

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /root/image-recognition-with-synthetic-training-data/model/models/mask_rcnn_vehicles-clean-start-5.h5


## Evaluation

In [9]:
# Sample 1000 to determine mean precision
image_ids = np.random.choice(dataset_val.image_ids, 1000)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))


mAP:  0.887


In [24]:
def score_test(directory='../../data/test/'):
    '''
    Run predict against all images in a directory and return a dataframe with predicted and actual classes
    '''
    preds = []
    for i, filename in enumerate(os.listdir(directory)):
        #print(filename)
        if(os.path.isdir(directory + '/' + filename)):
            continue
        img = Image.open(directory + '/' + filename)
        img = img.resize((256, 256))
        img_as_arr = np.array(img)
        results = model.detect([img_as_arr], verbose=0)
        if('car' in filename):
            actual_class = 1
        elif('pickup' in filename):
            actual_class = 2
        else:
            actual_class = 0
        if(len(results[0]['class_ids']) > 0):
            class_id = results[0]['class_ids'][0]
            score = results[0]['scores'][0]
            preds.append([filename, actual_class, class_id, float(score)])
        else:
            preds.append([filename, actual_class, 0, 0])
    return preds
preds_new2 = score_test()

In [25]:
df = pd.DataFrame(preds_new2)
confusion_matrix(df[1], df[2])

array([[ 1,  8,  1],
       [ 5, 32, 12],
       [ 6, 35,  9]])

In [16]:
print(classification_report(df[1], df[2]))

              precision    recall  f1-score   support

           0       1.00      0.57      0.73        21
           1       0.43      0.73      0.54        44
           2       0.41      0.20      0.27        44

   micro avg       0.49      0.49      0.49       109
   macro avg       0.61      0.50      0.51       109
weighted avg       0.53      0.49      0.47       109

